In [77]:
# default_exp client

In [91]:
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [92]:
from private import server_vars

In [93]:
base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

In [94]:
import requests

#from yx_motor import logger, config, version

default_headers = {"Content-Type": "application/json",
                    "Accept": "*/*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip,deflate"
                  }


class API(object):
    def __init__(self, api_url, headers=None):
        self.api_url = api_url
        headers = headers or default_headers
        self.headers = headers.copy()
        self.jar = requests.cookies.RequestsCookieJar()

    def get_path(self, url):
        api_url = self.api_url if not self.api_url.endswith("/") else self.api_url[:-1]
        template = "{}{}" if url.startswith("/") else "{}/{}"
        return template.format(api_url, url)

    def post(self, url, json=None, params=None, files=None, data=None, cookies=None, verify=False):
        if not cookies:
            cookies = self.jar
        path = self.get_path(url)
#         logger.debug("POST request sent to: {} \n\theaders: {}\n\tjson: {}\n\tparams: {}\n\tfiles: {}\n\tdata: {}"
#                      .format(path, self.headers, json, params, files, data))
        response = requests.post(path, 
                                 json=json, 
                                 params=params, 
                                 headers=self.headers, 
                                 files=files, 
                                 data=data, 
                                 cookies=cookies, 
                                 verify=verify)
#         logger.debug("Response status code: {}".format(response.status_code))
#         logger.debug("Response content: {}".format(response.content))
        return response

    def put(self, url, json=None, params=None):
        path = self.get_path(url)
        logger.debug("PUT request sent to: {} \n\theaders: {}\n\tjson: {}\n\tparams: {}"
                     .format(path, self.headers, json, params))
        response = requests.put(path, json=json, params=params, headers=self.headers)
#         logger.debug("Response status code: {}".format(response.status_code))
#         logger.debug("Response content: {}".format(response.content))
        return response

    def get(self, url, json=None, params=None, files=None, data=None, cookies=None, verify=False):
        if not cookies:
            cookies = self.jar
        path = self.get_path(url)
#         logger.debug("GET request sent to: {} \n\theaders: {}\n\tjson: {}\n\tparams: {}"
#                      .format(path, self.headers, json, params))
        response = requests.get(path, 
                                params=params, 
                                headers=self.headers, 
                                json=json,
                                files=files, 
                                data=data, 
                                cookies=cookies, 
                                verify=verify)
#         logger.debug("Response status code: {}".format(response.status_code))
#         logger.debug("Response content: {}".format(response.content))
        return response

    def delete(self, url, json=None, params=None):
        path = self.get_path(url)
        response = requests.delete(path, params=params, headers=self.headers, json=json)
#         logger.debug("DELETE request sent to: {} \n\theaders: {}\n\tjson: {}\n\tparams: {}"
#                      .format(response.url, self.headers, json, params))
#         logger.debug("Response status code: {}".format(response.status_code))
#         logger.debug("Response content: {}".format(response.content))
        return response

In [95]:
class Jobs:
    "class for all supported jobs APIs"
    
    
    def __init__(self,api):
        self.api=api
        self.base_endpoint='jobs/'

    def get_job(self,jobid=''):
        response = self.api.get(url=f'{self.base_endpoint}{jobid}')
        return response

    def get_log(self,jobid):
        endpoint='/logs'
        response = self.api.get(url=f'{self.base_endpoint}{jobid}{endpoint}')
        return response
    def cancel_job(self,jobid):
        endpoint='/cancel'
        response = self.api.post(url=f'{self.base_endpoint}{jobid}{endpoint}')

In [96]:
# export
import requests

class Motor:
    "Wrapper for Alteryx Server API."
    
    def __init__(self, 
                 base_url: str, 
                 login_email: str, 
                 login_pwd: str):
        
        self.base_url = base_url
        self.api_url = f"{base_url}api/v1/"
        
        self.auth_endpoint = "authenticate"
        
        self.login_email = login_email
        self.login_pwd = login_pwd
        
        self.headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate"
        }
        
        self.api = API(api_url=self.api_url, 
                       headers=self.headers)
        
        self.jobs = Jobs(self.api)
        
        self.is_authenticated = False
        self.authenticate()
        
        
    def authenticate(self) -> requests.Response:
        payload = {
            "email": self.login_email, 
            "password": self.login_pwd
        }
        response = self.api.post(url=self.auth_endpoint, 
                                 json=payload)
        if response.status_code == 200:
            self.api.jar.update(response.cookies)
            self.is_authenticated = True
        return response

    def get_users(self):
        response = self.api.get("users")
        return response


In [97]:
motor = Motor(base_url,
              login_email, 
              login_pwd)

In [98]:
motor.is_authenticated

True

In [47]:
response2 = motor.get_users()
y = response2.json()
for i in y[:10]:
    print(i['id'])

0807afc8-b47e-4092-975c-ea5da1035ed5
0e3398bc-4da4-4560-92ae-5731a68bc7e7
1a238ac4-382c-4e92-9ef6-90fe42d31deb
1abed64e-ff5a-4546-bc13-98076e936065
1d6bbcfe-b013-4068-8bfe-b9c992402fa7
1f8740ac-9bb0-4feb-9193-af79827111e1
26008c9f-8c18-4640-a549-d13b16c8441a
26795f67-615f-4d96-8d4f-6d32197ed2e9
26b437e9-76c5-43e1-a114-df989f393f95
2da81d57-24e7-411d-aef4-e8bbfec4bc98


In [48]:
Jobs(api=motor.api).get_job('a829c8c6-6143-4a95-aafa-332d842bd210').json()

{'requestUuid': '564f59df-23b4-452a-86c8-cd542fbda943',
 'errors': [{'code': '',
   'message': 'JobOrchestrator.getJob() with args: [ { "requestUuid": "564f59df-23b4-452a-86c8-cd542fbda943","auditEvents": "[  ]","privateSession": "{ NESTED OBJECT }","privateSiteId": "bbb722b8-b902-4371-a87a-a8c79cbf1984","clientIp": "::ffff:192.168.63.132" },"a829c8c6-6143-4a95-aafa-332d842bd210", { "schema": "{ NESTED OBJECT }","schemaErrorCode": "{ NESTED OBJECT }","from": 0,"pageSize": 10,"sortBy": "[  ]","filterStrategy": "AND" }]',
   'debugInfo': 'AepError: JobOrchestrator.getJob() with args: [ { "requestUuid": "564f59df-23b4-452a-86c8-cd542fbda943","auditEvents": "[  ]","privateSession": "{ NESTED OBJECT }","privateSiteId": "bbb722b8-b902-4371-a87a-a8c79cbf1984","clientIp": "::ffff:192.168.63.132" },"a829c8c6-6143-4a95-aafa-332d842bd210", { "schema": "{ NESTED OBJECT }","schemaErrorCode": "{ NESTED OBJECT }","from": 0,"pageSize": 10,"sortBy": "[  ]","filterStrategy": "AND" }]\n    at makeError (

In [108]:
req1 = motor.jobs.get_job()

req1.json()

#dir(req1)

{'jobs': [{'jobId': '845ace15-cfca-46f6-89c5-83cdf9c217a0',
   'workerId': '3b0d8827-c3a9-4e1d-ba77-304d339c71a6',
   'scheduleId': 'c68a62f6-24dd-47b6-ac5c-840323d0f75b',
   'executionOrdinal': 1,
   'queuedDate': '2020-04-07T14:01:11.890Z',
   'scheduledStartDate': '2020-04-07T14:01:11.890Z',
   'actualStartDate': '2020-04-07T14:01:12.741Z',
   'completionDate': '2020-04-07T14:02:18.607Z',
   'runTime': 66,
   'status': 'completed',
   'result': 'warning',
   'siteId': 'bbb722b8-b902-4371-a87a-a8c79cbf1984',
   'creationDate': '2020-04-07T14:01:11.879Z',
   'lastUpdate': '2020-04-07T14:02:18.595Z',
   'assetVersion': 1,
   'retryCount': 0,
   'notes': None,
   'priority': 50,
   'jobNo': 9,
   'name': 'oracle_loader.yxzp',
   'userId': 'e1dbac67-cefb-4acb-aeb6-6761b0e41c3e',
   'assetId': 'ce9ab2b9-353b-44f2-a825-45b3f40ce058',
   'type': 'immediate',
   'frequencyInterval': 'manual',
   'outputs': []},
  {'jobId': '5bb5bfc3-dc33-4828-b38d-3023370e5746',
   'workerId': '3b0d8827-c3a9

In [37]:
y = req1.json()
for i in y['jobs']:
    print(motor.jobs.get_log(i['jobId']).json())

TypeError: 'Mock' object is not subscriptable

In [90]:
#hide
from unittest.mock import Mock
from unittest.mock import patch
from unittest import mock
class TestJobs(TestCase):
    @patch('Motor.jobs')
    def test_get_job(self, Mock):
            get_job = MockGetJobs()

            get_job.return_value = [
                {
                    'userId': 1,
                    'id': 1,
                    'title': 'Test Title',
                    'body': 'Far out in the uncharted backwaters of the unfashionable  end  of the  western  spiral  arm  of  the Galaxy\ lies a small unregarded yellow sun.'
                }
            ]

            response = get_job()
            self.assertIsNotNone(response)
            self.assertIsInstance(response[0], dict)


NameError: name 'TestCase' is not defined

In [89]:
motor.jobs.get_job()

{'jobs': [{'jobId': '845ace15-cfca-46f6-89c5-83cdf9c217a0',
   'workerId': '3b0d8827-c3a9-4e1d-ba77-304d339c71a6',
   'scheduleId': 'c68a62f6-24dd-47b6-ac5c-840323d0f75b',
   'executionOrdinal': 1,
   'queuedDate': '2020-04-07T14:01:11.890Z',
   'scheduledStartDate': '2020-04-07T14:01:11.890Z',
   'actualStartDate': '2020-04-07T14:01:12.741Z',
   'completionDate': '2020-04-07T14:02:18.607Z',
   'runTime': 66,
   'status': 'completed',
   'result': 'warning',
   'siteId': 'bbb722b8-b902-4371-a87a-a8c79cbf1984',
   'creationDate': '2020-04-07T14:01:11.879Z',
   'lastUpdate': '2020-04-07T14:02:18.595Z',
   'assetVersion': 1,
   'retryCount': 0,
   'notes': None,
   'priority': 50,
   'jobNo': 9,
   'name': 'oracle_loader.yxzp',
   'userId': 'e1dbac67-cefb-4acb-aeb6-6761b0e41c3e',
   'assetId': 'ce9ab2b9-353b-44f2-a825-45b3f40ce058',
   'type': 'immediate',
   'frequencyInterval': 'manual',
   'outputs': []},
  {'jobId': '5bb5bfc3-dc33-4828-b38d-3023370e5746',
   'workerId': '3b0d8827-c3a9